In [3]:
import numpy as np
from numpy.linalg import det ,inv
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator

import sys
# setting path
sys.path.append('/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/vdw')
from genetic import Gen1

import random
random.seed(99)

In [ ]:
a1 = [1,0]; a2=[0,1];
A  = np.transpose([a1,a2]);

theta =  np.pi/4;
R = [ [ np.cos(theta), -np.sin(theta)], [ np.sin(theta),  np.cos(theta)] ];

b1 = np.sqrt(2)*np.dot(R,a1);  b2 = a2 #np.sqrt(2)*np.dot(R,a2);
B  = np.transpose([b1,b2]);

a = A#np.array([[0,1],[1,0]])
b = B#np.array([[0,5],[5,0]])

In [ ]:

eps = 0.0001

bond=20

def tAtB(params):
    tA = np.array([[params[0],params[1]],
                   [params[2],params[3]]])
    
    tB = np.dot(np.dot(tA,a), inv(b)) # tAa=tBb
    
    return tA, tB
          
def fit_fucntion(params):
          
        tA, tB= tAtB(params)
        tAa= np.dot(tA,a)
        tBb= np.dot(tB,b)
        
        # main condition
        zero_mat= tAa - tBb
        s=0
        for row in zero_mat:
            for e in row:
                s+=e*e
        
        # mimimum TA
        detTAa= det(tAa)*det(tAa) # minimum but biger than 0
        detTBb= det(tBb)*det(tBb)
        
        # TB integer 
        cons =9999999
        tB_con = 0  
        for row in tB:
            for e in row:
                tB_con += ((round(e)-e))*((round(e)-e)) # e*e
        tB_con = tB_con*cons
        
        
        f =  1/detTAa + tB_con**2 # s + (2-detTAa)*(2-detTAa)*10 + 1/detTAa + tB_con+1/detTBb # since we are working with integer the smallest pos det is 1
        
        return f

In [ ]:
# Experiment settings

#Hyperparameters of the model
model_par = {
    
    'cell_split_number': 4, 
    'subjects_in_cell': 3,

    'nr_clones': 10,
    'mutation_gaussian_sd': 4,

    'pins': 7,
    'gene_quality': 1,
    
}




input_size = 4 # P=[tA00,tA01,tA10,tA11]

bounds =[[-bond,bond]for _ in range(input_size)] # searching intervals for the solution

# Experiment
experiment = Gen1(fit_fucntion ,input_size, bounds, model_par)

In [ ]:
## Automate evolution

nr_epochs = 100
last_generation = experiment.evolve(nr_epochs)
fit = [fit_fucntion(x) for x in last_generation] 
print("fit:",fit)


In [ ]:
best =last_generation [fit.index(min(fit))]
print(best)

In [ ]:
P = best #[1, 1, 0, 1]#[2, 0, 0, -1]#[0, -1, -1, 0]#[-1, 1, 0, 1]
tA,tB = tAtB(P)
zero_mat= np.dot(tA,a) - np.dot(tB,b)
print("\n _____\n tA:\n",tA)
print("det:",det(tA))
print("\n _____\n tB:\n",tB)
print("det:",det(tB))
print("\n zero_mat:\n",zero_mat)